In [1]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
import sys
# !{sys.executable} -m pip install PyQt5
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
os.environ["OMP_NUM_THREADS"] = "12"
from scipy.optimize import minimize
# Qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit import Parameter, ParameterVector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeQuebec

In [2]:
def mitarai(quantumcircuit,features, num_wires):
    # encoding as proposed by Mitarai et al.
    num_features = len(features)

    for i in range(num_wires):
        feature_idx = i % num_features  # Calculate the feature index using modulo
        if np.isnan(np.arcsin(features[feature_idx])) or np.isnan(np.arccos(features[feature_idx])):
            print(f'Ignoring NaN found at index: {feature_idx}. With feature: {features[feature_idx]}. ')
        else:
            quantumcircuit.ry(np.arcsin(features[feature_idx]), i)
            quantumcircuit.rz(np.arccos(features[feature_idx] ** 2), i)



def entangle_cnot(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cnot gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cx(i, 0)
        else:
            quantumcircuit.cx(i, i+1)


def entangle_cz(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cz gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cz(i, 0)
        else:
            quantumcircuit.cz(i, i+1)


def HardwareEfficient(quantumcircuit,num_wires):
    parameters =ParameterVector('x',num_qubits*3)
    for qubit in range(num_qubits):
        quantumcircuit.rx(parameters[qubit * 3], qubit)  
        quantumcircuit.rz(parameters[qubit * 3 + 1], qubit)  
        quantumcircuit.rx(parameters[qubit * 3 + 2], qubit)  
    entangle_cnot(quantumcircuit,num_wires)



In [3]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    # print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [4]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [5]:
def circuit(features,nqubits):
    qc = QuantumCircuit(nqubits)
    mitarai(qc,features,nqubits)
    entangle_cz(qc,nqubits)
    qc.barrier()
    mitarai(qc,features,nqubits)
    entangle_cz(qc,num_qubits)
    qc.barrier()
    HardwareEfficient(qc,nqubits)
    qc.barrier()
    return qc

In [6]:
with open('PCA5_0.8_Morgan_train.bin','rb') as f:
    train = joblib.load(f)

with open('PCA5_0.8_Morgan_test.bin','rb') as f:
    test = joblib.load(f)

with open('PCA5_0.8_Morgan_scaler.bin','rb') as f:
    scaler = joblib.load(f)

X_train, y_train = train['X'],train['y']
X_test, y_test = test['X'],test['y']

In [7]:
num_qubits = 5
generator = np.random.default_rng(12958234)
angles = generator.uniform(-np.pi, np.pi, num_qubits*3).reshape(-1,3)







MinMaxScaler(feature_range=(-1, 1))

In [ ]:
weights = angles.reshape(-1,)
for x,y in zip(X_train,scaler.inverse_transform(y_train.reshape(-1,1)).flatten()):
    qc = circuit(x,num_qubits)
    # qc = qc.assign_parameters(dict(zip(qc.parameters,angles.flatten())))
    observables_labels = ''.join(['I']*(num_qubits-1))+"Z"
    observables = [SparsePauliOp(observables_labels)]
    
    estimator = StatevectorEstimator()
    
    mapped_observables = [observable.apply_layout(qc.layout) for observable in observables]
    # job = estimator.run([(qc, mapped_observables, angles.flatten())])
    # job.result()[0].data.evs
    
    res = minimize(cost_func,weights,args=(qc, mapped_observables, estimator),method="cobyla",)
    weights = res.x 
    print(res.fun,y)

In [ ]:
def fit(parameters,features,target,nqubits):

    qc = circuit(features,nqubits)
    qc = qc.assign_parameters(dict(zip(qc.parameters,parameters.flatten())))
    observables_labels = ''.join(['I']*(nqubits-1))+"Z"
    observables = [SparsePauliOp(observables_labels)]
    
    estimator = StatevectorEstimator()
    
    mapped_observables = [observable.apply_layout(qc.layout) for observable in observables]
    job = estimator.run([(qc, mapped_observables)])
    
    
    print(target)
    res = minimize(cost_func,angles.reshape(-1,),args=(qc, mapped_observables, estimator),method="cobyla",)
    print(res)

In [ ]:
[fit(angles.flatten(),x,y,num_qubits) for x,y in zip(X_train,y_train)]

In [ ]:

# qiskit_statevector_results = job.result()[0].data['evs']
# print(qiskit_statevector_results)